# Heads-Up

In the notebook, we are working on generating
graphs whose nodes have Euclidean coordinates;
One thing to pay attention to is, in real world
road network, nodes (intersections) that are close
to each other geographically are more likely to be
connected with edges (roads).

Edge centrality will be added as part of the heuristic;
meantime, cosine distance can also be a good heuristic;
The geographical coordinates of nodes can be included.

# Questions

Does centrality really encode information about your destination?

# Think with Analogy

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import random
import numpy as np
import mxnet as mx
import logging
from sklearn.metrics import accuracy_score
from utils import plot_g

In [ ]:
NUM_NODE = 100
WEIGHT_MIN = .0
WEIGHT_MAX = 1.

In [ ]:
def calc_euclidean_dist(G, tmp_node, node):
    
    p_1 = np.array([G.nodes[tmp_node]['x'], G.nodes[tmp_node]['y']])
    p_2 = np.array([G.nodes[node]['x'], G.nodes[node]['y']])
    return np.sqrt(np.sum((p_1 - p_2)**2))

def generate_low_degree_g(node_size=20, min_out_degree=2, max_out_degree=4, weight_min=WEIGHT_MIN, weight_max=WEIGHT_MAX):
    
    G = nx.Graph()
    
    grid_size = 1000
    euclidean_coords = np.linspace(0.01, 1.0, num=grid_size, endpoint=False)
    coords_indices = list(range(grid_size))
    
    random.shuffle(coords_indices)
    x_coords = euclidean_coords[coords_indices][:node_size]
#     print(x_coords)
    random.shuffle(coords_indices)
    y_coords = euclidean_coords[coords_indices][:node_size]
#     print(y_coords)
    
    # Add coordinates to nodes
    for node, coord in enumerate(zip(x_coords, y_coords)):
#         print(node, coord[0], coord[1])
        G.add_node(node, x=coord[0], y=coord[1])
    
    for node in G.nodes:
        
        tmp_nodes = list(G.nodes)
        tmp_nodes.remove(node)
        node_dist = map(lambda tmp_node: (tmp_node, calc_euclidean_dist(G, tmp_node, node)), tmp_nodes)
        node_dist = sorted(node_dist, key=lambda d:d[1])
        
        num_of_neighbors = random.randint(min_out_degree, max_out_degree)
#         print(node, out_neighbors)
        
#         G.add_edges_from(map(lambda d:(node, d), out_neighbors))
        
        for tmp_node in node_dist:

            if G.degree(tmp_node[0]) >= max_out_degree \
                or G.degree(node) >= num_of_neighbors:
                # This node has maximum number of neighbors already
                continue
            
            weight = random.uniform(weight_min, weight_max)
            G.add_edge(node, tmp_node[0], weight=weight)
    
    # Add centrality to edges
    edge_centrality = nx.edge_betweenness_centrality(G, \
                                                     k=G.number_of_nodes(), \
                                                     weight='weight')
    assert len(edge_centrality) == G.number_of_edges()
    
    for edge_data in G.edges.data():
        edge_data[2]['centrality'] = edge_centrality[(edge_data[0], edge_data[1])]
        
    return G

In [ ]:
G = generate_low_degree_g()
max_degree = max(G.degree, key=lambda d: d[1])[1]
min_degree = min(G.degree, key=lambda d: d[1])[1]
print(max_degree)
print(min_degree)
print(G.number_of_nodes())
print(G.number_of_edges())
plot_g(G)

In [ ]:
def extract_path(prev, src, dst):
    
    path = []
    u = dst

    while prev[u] != src:
        path.insert(0, u)
        u = prev[u]
    
    path.insert(0, u)
    path.insert(0, src)
    return path

def bfs_beam_edges_path(G, src, dst):
    
    centrality = nx.edge_betweenness_centrality(G, weight='weight', \
                                              k=G.number_of_nodes(), \
                                            normalized=True)
    
    prev = [-1 for _ in range(G.number_of_nodes())]
    
    for u, v in nx.bfs_beam_edges(G, src, value=centrality.get):
        print(u, v, G.get_edge_data(u, v)['centrality'], G.get_edge_data(u, v)['weight'])
        prev[v] = u

    return extract_path(prev, src, dst)

for src in G.nodes():
    for dst in G.nodes():
        
        if src == dst:
            continue
        
        print(nx.dijkstra_path(G, src, dst, weight='weight'))
        print(bfs_beam_edges_path(G, src, dst))
        print('')

In [ ]:
tmp = np.zeros((2 ,3))
tmp[:, -1] = 2
tmp[0, -1] == 2

In [ ]:
def calc_cosine_dist(p1, p2):
    return np.dot(p1, p2) / (np.linalg.norm(p1) * np.linalg.norm(p2))

def encode_edges(G, node, dst):
    
    ret = np.zeros((max_degree, 6))
    ret[:, -1] = -1
    
    for idx, edge in enumerate(G.edges(node)):
        u = edge[0]
        v = edge[1]
        
        ret[idx][0] = G.get_edge_data(u, v)['centrality']
        ret[idx][1] = G.get_edge_data(u, v)['weight']
        
        x_u = G.nodes[u]['x']
        y_u = G.nodes[u]['y']
        x_v = G.nodes[v]['x']
        y_v = G.nodes[v]['y']
        x_dst = G.nodes[dst]['x']
        y_dst = G.nodes[dst]['y']
        
        ret[idx][2] = calc_cosine_dist((x_v-x_u, y_v-y_u), (x_dst-x_u, y_dst-y_u))
        ret[idx][3] = x_v
        ret[idx][4] = y_v
        ret[idx][-1] = v
        
    return (ret, x_dst, y_dst)

def generate_dataset(G):
    
    X = []
    y = []

    for paths in nx.all_pairs_dijkstra_path(G):

        print("src :%d" % paths[0])
        print(paths[1])
        print('')
        src = paths[0]

        for dst, path in paths[1].items():

            if dst == src:
                continue

            # X = cur_node + dst_node
            # y = next_node (Dijkstra)

            cur_node = src
            for mid_node in path[1:]:

                print('X:(%d, %d), y:(%d)' % (cur_node, dst, mid_node))
                X.append(encode_edges(G, cur_node, dst))
                y.append(mid_node)

                cur_node = mid_node

    #         print(cur_node, dst)
            print(path)
            print('')
    
    return X, y

In [ ]:
features, labels = generate_dataset(G)

In [ ]:
def find_label_idx(feature, label):
    
    for idx, row in enumerate(feature):
        if label == row[-1]:
            break
    
    return idx

def augment_dataset(X, y, augmentation_index=20):
    
    X_aug, y_aug = [], []
    indices = np.arange(max_degree)
    
    for feature, label in zip(X, y):
        for _ in range(augmentation_index):
            np.random.shuffle(indices)
            
            feature_tmp = feature[0][indices]
            label_tmp = find_label_idx(feature_tmp, label)
            X_aug.append(np.append(feature_tmp[:, :-1].reshape(-1), [feature[1], feature[2]]))
            y_aug.append(label_tmp)
            
    return np.array(X_aug), np.array(y_aug)

In [ ]:
X, y = augment_dataset(features, labels)

In [ ]:
X.shape